In [337]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier

In [338]:
train = pd.read_csv(r'C:\Users\Rahul\Desktop\Cardiac-Risk-Prediction\Train.csv')
test = pd.read_csv(r'C:\Users\Rahul\Desktop\Cardiac-Risk-Prediction\Test.csv')
sample = pd.read_excel(r'C:\Users\Rahul\Desktop\Cardiac-Risk-Prediction\Sample_Submission.xlsx')

In [339]:
train.head()

,Gender,Chain_smoker,Consumes_other_tobacco_products,HighBP,Obese,Diabetes,Metabolic_syndrome,Use_of_stimulant_drugs,Family_history,History_of_preeclampsia,CABG_history,Respiratory_illness,UnderRisk
0,1,1,1,0,1,0,0,0,1,0,0,0,no
1,1,0,1,0,1,0,0,0,1,0,0,0,no
2,1,0,1,0,1,0,0,0,1,0,0,0,no
3,1,0,1,0,1,0,0,0,1,0,0,0,no
4,1,0,0,0,0,0,1,1,0,0,0,0,no


In [340]:
test.head()

,Gender,Chain_smoker,Consumes_other_tobacco_products,HighBP,Obese,Diabetes,Metabolic_syndrome,Use_of_stimulant_drugs,Family_history,History_of_preeclampsia,CABG_history,Respiratory_illness
0,1,0,1,1,1,0,0,0,1,0,0,0
1,1,0,1,0,1,0,0,0,1,0,0,0
2,1,1,1,0,1,0,0,0,1,0,0,0
3,2,0,0,0,1,0,0,0,1,0,0,0
4,1,0,1,0,0,0,0,1,1,0,0,0


In [341]:
train.describe()

,Gender,Chain_smoker,Consumes_other_tobacco_products,HighBP,Obese,Diabetes,Metabolic_syndrome,Use_of_stimulant_drugs,Family_history,History_of_preeclampsia,CABG_history,Respiratory_illness
count,889.000000,889.000000,889.00000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,1.258718,0.120360,0.83802,0.086614,0.919010,0.055118,0.042745,0.082115,0.925759,0.017998,0.021372,0.032621
std,0.500557,0.325565,0.36864,0.281427,0.272973,0.228339,0.202395,0.274694,0.262310,0.133018,0.144704,0.177742
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [342]:
test.describe()

,Gender,Chain_smoker,Consumes_other_tobacco_products,HighBP,Obese,Diabetes,Metabolic_syndrome,Use_of_stimulant_drugs,Family_history,History_of_preeclampsia,CABG_history,Respiratory_illness
count,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000
mean,1.295812,0.120419,0.793194,0.054974,0.892670,0.052356,0.060209,0.107330,0.924084,0.005236,0.020942,0.031414
std,0.511221,0.325878,0.405547,0.228228,0.309938,0.223036,0.238186,0.309938,0.265211,0.072262,0.143379,0.174662
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [343]:
train.isnull().sum()

Gender                             0
Chain_smoker                       0
Consumes_other_tobacco_products    0
HighBP                             0
Obese                              0
Diabetes                           0
Metabolic_syndrome                 0
Use_of_stimulant_drugs             0
Family_history                     0
History_of_preeclampsia            0
CABG_history                       0
Respiratory_illness                0
UnderRisk                          0
dtype: int64

In [344]:
test.isnull().sum()

Gender                             0
Chain_smoker                       0
Consumes_other_tobacco_products    0
HighBP                             0
Obese                              0
Diabetes                           0
Metabolic_syndrome                 0
Use_of_stimulant_drugs             0
Family_history                     0
History_of_preeclampsia            0
CABG_history                       0
Respiratory_illness                0
dtype: int64

In [345]:
list(train.columns[:-1])

['Gender',
 'Chain_smoker',
 'Consumes_other_tobacco_products',
 'HighBP',
 'Obese',
 'Diabetes',
 'Metabolic_syndrome',
 'Use_of_stimulant_drugs',
 'Family_history',
 'History_of_preeclampsia',
 'CABG_history',
 'Respiratory_illness']

In [346]:
train.Gender.value_counts()

1    607
2    256
0     26
Name: Gender, dtype: int64

In [347]:
train.head(1)

,Gender,Chain_smoker,Consumes_other_tobacco_products,HighBP,Obese,Diabetes,Metabolic_syndrome,Use_of_stimulant_drugs,Family_history,History_of_preeclampsia,CABG_history,Respiratory_illness,UnderRisk
0,1,1,1,0,1,0,0,0,1,0,0,0,no


In [348]:
train.groupby(['Chain_smoker','Consumes_other_tobacco_products','HighBP'])['UnderRisk'].value_counts()

Chain_smoker  Consumes_other_tobacco_products  HighBP  UnderRisk
0             0                                0       no            83
                                                       yes           15
                                               1       no             8
                                                       yes            6
              1                                0       no           482
                                                       yes          141
                                               1       no            25
                                                       yes           22
1             0                                0       no            25
                                                       yes            1
                                               1       no             6
              1                                0       no            62
                                                       yes            3

In [349]:
train.CABG_history.value_counts()

0    870
1     19
Name: CABG_history, dtype: int64

In [350]:
train['Total_Risk']= np.sum(train.loc[:,'Chain_smoker':'Respiratory_illness'] , axis = 1)
test['Total_Risk']= np.sum(test.loc[:,'Chain_smoker':'Respiratory_illness'] , axis = 1)

In [351]:
tr_dummies = pd.DataFrame(pd.get_dummies(train[['Gender','Total_Risk']].astype('category')))
te_dummies = pd.DataFrame(pd.get_dummies(test[['Gender','Total_Risk']].astype('category')))

train = pd.concat([train,tr_dummies],axis=1)
test = pd.concat([test,te_dummies],axis=1)

train.drop(columns=['Gender','Total_Risk'],inplace=True)
test.drop(columns=['Gender','Total_Risk'],inplace=True)

In [352]:
label = train.UnderRisk
label = label.apply(lambda x : 1 if (x=='yes') else 0)
train.drop(columns=['UnderRisk'],inplace=True)

In [353]:
label.value_counts()

0    699
1    190
Name: UnderRisk, dtype: int64

In [354]:
def model(m, train, label, test, random):
    strat = StratifiedShuffleSplit(n_splits= 9, random_state = random)
    for tr_index,te_index in strat.split(train,label):
        
        xtrain,xval = train.loc[tr_index,:],train.loc[te_index,:]
        ytrain,yval = label.loc[tr_index], label.loc[te_index]
        
        m.fit(xtrain,ytrain)
        tr_pred = m.predict_proba(xtrain)
        te_pred = m.predict_proba(xval)
        
        print('Training Loss :{}                                    Testing Loss : {}'.format(log_loss(ytrain,tr_pred),log_loss(yval,te_pred)))
    return(pd.DataFrame(m.predict_proba(test)))    

In [230]:
model(RandomForestClassifier(),train,label,test,78)

Training Loss :0.43089295384379633                                    Testing Loss : 1.3161731740421474
Training Loss :0.4343372732867356                                    Testing Loss : 1.2410895426826134
Training Loss :0.44049043071648414                                    Testing Loss : 0.4886512618911808
Training Loss :0.44277092497604725                                    Testing Loss : 0.46354525693215937
Training Loss :0.4455177197984341                                    Testing Loss : 0.4585640323511888


In [231]:
model(DecisionTreeClassifier(),train,label,test,578)

Training Loss :0.4355273894604451                                    Testing Loss : 0.5035072623524859
Training Loss :0.4386275196999236                                    Testing Loss : 0.462685646322105
Training Loss :0.4392165776303231                                    Testing Loss : 0.8307890880332518
Training Loss :0.4355402011947112                                    Testing Loss : 1.2539385475740754
Training Loss :0.4380461624350464                                    Testing Loss : 1.2391058833087707


In [358]:
result = model(LogisticRegression(random_state=8214),train,label,test,82341)

Training Loss :0.4622574614754352                                    Testing Loss : 0.491314523406453
Training Loss :0.462515796362939                                    Testing Loss : 0.5091778153173262
Training Loss :0.4655074243236139                                    Testing Loss : 0.4658588204914032
Training Loss :0.4662927914262439                                    Testing Loss : 0.4694092806849073
Training Loss :0.46411499210551965                                    Testing Loss : 0.4798158989820729
Training Loss :0.465251122797637                                    Testing Loss : 0.47316291041149905
Training Loss :0.46327989066287273                                    Testing Loss : 0.4870554816932942
Training Loss :0.4631010747723295                                    Testing Loss : 0.48883732806633273
Training Loss :0.46544246092296604                                    Testing Loss : 0.4648984225412641


In [233]:
model(AdaBoostClassifier(),train,label,test,5098)

Training Loss :0.6513441373206096                                    Testing Loss : 0.6630103815598669
Training Loss :0.6523611622574397                                    Testing Loss : 0.6472876802756348
Training Loss :0.6518827728941866                                    Testing Loss : 0.6500419854423792
Training Loss :0.652928449922287                                    Testing Loss : 0.6494571792102924
Training Loss :0.6517709994839674                                    Testing Loss : 0.6588641513448045


In [234]:
model(GradientBoostingClassifier(),train,label,test,238)

Training Loss :0.44629148001058055                                    Testing Loss : 0.47978934936704104
Training Loss :0.44298238170011817                                    Testing Loss : 0.5051486930582938
Training Loss :0.44173601016457087                                    Testing Loss : 0.504117719910796
Training Loss :0.4486326032558914                                    Testing Loss : 0.42587687487204234
Training Loss :0.44347422142606363                                    Testing Loss : 0.4734679388283704


In [235]:
model(ExtraTreesClassifier(),train,label,test,520978)

Training Loss :0.4384171417919913                                    Testing Loss : 1.1966157236087642
Training Loss :0.4425891344064354                                    Testing Loss : 0.4410146365713371
Training Loss :0.43695259472550196                                    Testing Loss : 0.8591189879084301
Training Loss :0.4327408184923422                                    Testing Loss : 1.260886849863909
Training Loss :0.4438900823374341                                    Testing Loss : 0.43743421038800556


In [236]:
model(SVC(probability=True),train,label,test,5278)

Training Loss :0.49981320264668405                                    Testing Loss : 0.5152536625811162
Training Loss :0.49880220479066467                                    Testing Loss : 0.5132638587171109
Training Loss :0.5065616855044063                                    Testing Loss : 0.5148960766268247
Training Loss :0.5020222700991029                                    Testing Loss : 0.5171888364519557
Training Loss :0.5129740242980795                                    Testing Loss : 0.5180489892611998


In [360]:
result.columns= sample.columns

In [362]:
result.to_excel('Base_lr.xlsx',index=False)